<a href="https://colab.research.google.com/github/MarinaWolters/Coding-Tracker/blob/master/CIS_5210_Txt2Img_Judgments.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Judge The Quality of AI Generated Images

This is an optional extra credit assignment for CIS 4210/5210.  In this assignment, you'll to judge the quality of images that were automatically generated by an AI system takes a text prompt as input and then generates images.  

There are several text2img generation systems that have come out recently and that have impressively good results.  If you'd like to check them out, I recommed looking at
* [DALL*E 2 from OpenAI](https://openai.com/dall-e-2/)
* [Stable Diffusion an open source text-to-image model from StabilityAI](https://huggingface.co/spaces/stabilityai/stable-diffusion) 
* [Midjourney, a commercial version from StabilityAI](https://www.midjourney.com/home/?callbackUrl=%2Fapp%2F)

In this EC assignment, we'll show you sets of images that depict the same underlying subject, and ask you to pick your favorite.  In the first cell, you'll see 8 sets of 4 images (move through the different ones with the rounds  slider bar).  You'll your favorite from each of these 8 sets.  After that we'll move on the semi-finals, where your favorites will be paired against each other, and then to the final round where you pick your favoite from the remaining 2 images.

After that, your annotations will be saved into a JSON file in your Google Drive. 

To get full extra credit, you should re-run this Python Notebook 25 times to annotate a total of 25 different sets of images.  To do so, just click the play button for each cell in the Notebook until you get to the end, confirm that you saved your current annotations, and then go back to the first cell and start again.

If you have any question, please email Rebecca Maryann Dsouza <rdsouza@seas.upenn.edu>.


# Start the Notebook

The notebook will ask for access to your Google Drive.  Please grant it so that we can save your annotations.

It'll save your annotations in a file in your Google Drive named with your Penn ID number followed by '_results.json'. 

In [ ]:
#@title Google Drive Setup
from google.colab import drive
drive.mount('/content/drive')

Next, we'll load the AI generated images.

In [ ]:
#@title Load AI Generated Images

import io
import os
import urllib
import json
import pandas as pd
from os import listdir
from os.path import isfile, join
from random import shuffle
import ipywidgets as widgets
from IPython.display import display
from ipywidgets import Box, VBox, HBox
from PIL import Image as pil_image
from PIL import PngImagePlugin as png_image


def prechecks():
    if not 'penn_id' in vars():
        penn_id = input("Please input your Penn ID number\n")
    json_file_name = "{0}_results.json".format(penn_id)
    path = '/content/drive/My Drive/'
    onlyfiles = [f for f in listdir(path) if isfile(join(path, f))]
    if json_file_name not in onlyfiles:
        with open(path+json_file_name, mode='w', encoding='utf-8') as f:
            json.dump([], f)
            print("File {0} has been created and should be uploaded to Gradescope!".format(json_file_name))
    else:
        print("Using existing file {0}, remember to upload it to Gradescope!".format(json_file_name))
    return path + json_file_name

def save_results_drive(entry):
    file_path_name = prechecks()
    data = []
    with open(file_path_name, mode='r', encoding='utf-8') as f:
        data = json.load(f)
        unique_items = []
        for item in data:
          subject = item['subject']
          if not subject in unique_items:
            print(subject)
            unique_items.append(subject)
        print("You have completed {num_completed} annotations previously.".format(num_completed=len(unique_items)))
    with open(file_path_name, mode='w', encoding='utf-8') as f:
        data.append(entry)
        json.dump(data, f)
        print("Added new annotation now!")
        print('\nTo annotate more items, go back to the Load AI Generated Images cell, and run it and all of the following cells.')



def get_cropped_image(file_n, grid_square):
    area = {
        0: (0,0,1024,1024), # WHOLE
        1: (0,0,512,512), #UPPER LEFT
        2: (512,0,1024,512), #UPPER RIGHT
        3: (0,512,512,1024), #LOWER LEFT
        4: (512,512,1024,1024) #LOWER RIGHT
    }
    return pil_image.open(file_n).crop(area[int(grid_square)])
    

def get_image_grid(imgs, rows, cols):
    assert len(imgs) == rows*cols

    w, h = imgs[0].size
    grid = pil_image.new('RGB', size=(cols*w, rows*h))
    grid_w, grid_h = grid.size
    
    for i, img in enumerate(imgs):
        grid.paste(img, box=(i%cols*w, i//cols*h))
    return grid


def get_image_bytes(img_data):
    imgByteArr = io.BytesIO()
    img_data.save(imgByteArr, format='PNG')
    imgByteArr = imgByteArr.getvalue()
    return imgByteArr


def get_image_widget(img_data_bytes):
    return widgets.Image(
            value=img_data_bytes,
            format='png',
            width=300,
            height=300,
        )
    

def get_pref_widget():
    return widgets.Dropdown(
        options=[('Pick the Best Image',0),('Upper Left', 1), ('Upper Right', 2), ('Lower Left', 3), ("Lower Right", 4)],
        value=0,
    )


def get_random_subject():
    subjects = ["parrot","table","book","fountain","bear","music","water","doctor","computer","apple","shark","dolphin","icecream","cake","engineer","teacher",
                "doll","house","shoes","cup","air sprite","alleyway","ancient hopi doll","android prince","angel","anubis","astronaut","back to the future",
                "beagle","blue jay","boris johnson","boxcar","buddha","cat","cathedral","centaur","chameleon","church","city","city street","cyborg","dancer"
                ,"dinosaurs in paris","dragon","elven female thief","emma watson","evil princess","fairy house","female warrior","fighter jet","gal gadot",
                "geisha","giant skull","girl leaving home with suitcases","gothic tower","hiking trail","house","jesus","joe biden","john f kennedy",
                "kathakali","knife","liminal space","man taking a selfie","mermaid","minion","mirror","mona lisa","moose head","necromancer",
                "necromancer sorceress","ninja frog","observation tower","octopus","possum","pyramids of giza","robot","rukia kuchiki","sacred fennec fox",
                "samurai robot","samurai tiger mask","sauron","sea monster","serpent","sheep","skull","sky","spongebob","stone steps","sunset",
                "supergirl","superman","temple of the sun","trump and Biden","two fishes","utopia","white wolf","wizard","wolf","yoda"
    ]
    shuffle(subjects)
    return subjects[0]

def get_random_trial():
    trials = [0,1,2,3]
    shuffle(trials)
    return trials[0]

CSV_PATH = "https://www.cis.upenn.edu/~ccb/data/StableDiffusionGPT/StableDiffusionGPT.csv"
FOLDER_PATH = "https://www.cis.upenn.edu/~ccb/data/StableDiffusionGPT"
SUBJECT = get_random_subject()
TRIAL = get_random_trial()

MOD_FOLDER = urllib.parse.quote(SUBJECT + "_" + str(TRIAL))
MOD_SUBJECT = urllib.parse.quote(SUBJECT)

VARIATION_PATHS = [
    "{0}/{1}/{2}_B.png".format(FOLDER_PATH, MOD_FOLDER, MOD_SUBJECT),
    "{0}/{1}/{2}_FS.png".format(FOLDER_PATH, MOD_FOLDER, MOD_SUBJECT),
    "{0}/{1}/{2}_A.png".format(FOLDER_PATH, MOD_FOLDER, MOD_SUBJECT),
    "{0}/{1}/{2}_K.png".format(FOLDER_PATH, MOD_FOLDER, MOD_SUBJECT),
    "{0}/{1}/{2}_A_K.png".format(FOLDER_PATH, MOD_FOLDER, MOD_SUBJECT),
    "{0}/{1}/{2}_FS_A.png".format(FOLDER_PATH, MOD_FOLDER, MOD_SUBJECT),
    "{0}/{1}/{2}_FS_K.png".format(FOLDER_PATH, MOD_FOLDER, MOD_SUBJECT),
    "{0}/{1}/{2}_FS_A_K.png".format(FOLDER_PATH, MOD_FOLDER, MOD_SUBJECT)
]

LABELS = [urllib.parse.unquote(_.split("/")[-1].split(".")[0]) for _ in VARIATION_PATHS]

img_widgets, pref_widgets, box_widgets = [],[],[]


print("Images are being generated for the subject '{0}'".format(SUBJECT))
for idx, path in enumerate(VARIATION_PATHS):
    try:
        urllib.request.urlretrieve(path, "temp.png")
        img_w = get_image_widget(get_image_bytes(get_cropped_image("temp.png",0)))
        pref_w = get_pref_widget()
        img_widgets.append(img_w)
        pref_widgets.append(pref_w)
    except:
        print("Unable to retrieve data, please run this cell again")
        break



# Initial Round

Please scroll through the 8 sets of images using the "Round" slider bar, and pick your favorite for each set.

In [ ]:
print("Please pick your favourite image of", SUBJECT)

In [ ]:
#@title Annotate 8 Sets
from ipywidgets import interact

@interact(Round=widgets.IntSlider(min=0, max=7, step=1, value=0))
def g(Round):
    if Round!=0 and pref_widgets[Round-1].value not in [1,2,3,4]:
      print("Please complete the previous image selection")
    else:
      display(Box([img_widgets[Round],pref_widgets[Round]]))

In [ ]:
#@title Semi-Finals
pref_A = get_pref_widget()
pref_B = get_pref_widget()
initial_preferences = {}
best_picks = []


done_eight = True

for _ in range(8):
  if pref_widgets[_].value not in [1,2,3,4]:
    done_eight = False

if not done_eight:
    print('You forgot to pick the best image in the previous examples, please complete those grid before executing this cell')
else:
    for idx, pw in enumerate(pref_widgets):
        initial_preferences[idx] = pw.value

    pref_img_data = []
    for idx, path in enumerate(VARIATION_PATHS):
        urllib.request.urlretrieve(path, "temp.png")
        pref_img_data.append(get_cropped_image("temp.png",initial_preferences[idx]))

    best_picks = [_ for _ in range(8)]
    shuffle(best_picks)
    subset_A = [pref_img_data[idx] for idx in best_picks[:4]]
    subset_B = [pref_img_data[idx] for idx in best_picks[4:]]

    img_w_A = get_image_widget(get_image_bytes(get_image_grid(subset_A,2,2)))
    img_w_B = get_image_widget(get_image_bytes(get_image_grid(subset_B,2,2)))
    display(VBox(children=[Box(children=[img_w_A, pref_A]),Box(children=[img_w_B, pref_B])]))

In [ ]:
#@title Finals
final_A, final_B = -1,-1
PREF = widgets.Dropdown(options=[('Which looks better?',0),('Left', 1), ('Right', 2), ('Left & Right both', 3)],value=0)
if pref_A.value not in [1,2,3,4] or pref_B.value not in [1,2,3,4]:
    print('You forgot to pick the best image in the previous two examples, please complete those grids before executing this cell')
else: 
    final_A = best_picks[pref_A.value-1]
    final_B = best_picks[pref_B.value+3]
    # print(final_A, final_B)
    IMGW = get_image_widget(get_image_bytes(get_image_grid([pref_img_data[final_A],pref_img_data[final_B]],1,2)))
    display(Box(children=[IMGW,PREF]))

In [ ]:
#@title Save your results
if PREF.value not in [1,2,3]:
    print('You forgot to pick the best image in the previous example, please complete that grid before executing this cell')
else:

    if not 'penn_id' in locals():
        penn_id = input("Please input your Penn ID number\n")

    df = pd.read_csv(CSV_PATH)
    result_entry = df.loc[(df['subject'] == SUBJECT) & (df['pno'] == TRIAL)].iloc[0,:].to_dict()
    for k,v in initial_preferences.items(): result_entry[LABELS[k]] = v
    result_entry['sf1'] = [LABELS[_] for _ in best_picks[:4]]
    result_entry['sf2'] = [LABELS[_] for _ in best_picks[4:]]
    result_entry['fn'] = [LABELS[final_A], LABELS[final_B]]
    result_entry['dc'] = PREF.value


    save_results_drive(result_entry)
    for _ in range(8): pref_widgets[_].value = 0
    pref_A.value = 0
    pref_B.value = 0
    PREF.value = 0